# 01. Library Load

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 02. Data Load

In [2]:
train =pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test =pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
sub =pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')

# 03. Data 결합하기

   - 똑같은 작업을 반복하지 않기 위해, train과 test 결합.
   - 단점 : train, test의 분포가 다르다면 오히려 좋지 않은 영향을 끼칠 수 있음. -> 우리 모델이 실수 할 수 있는 부분을 찾아줘야 함.

In [3]:
all_data=pd.concat([train,test])
all_data.head()

### columns 설명
- datetime : hourly date + timestamp
- season : 1=spring, 2=summer, 3=fall, 4=winter
- holiday : whether the day is considered a holiday(휴일 : 1, 휴일 X : 0)
- workingday : whether the day is neither a weekend nor holiday(주말, 휴일 아니면 : 1, 주말, 휴일 : 0)
- weather : 1 : Clear, Few clouds, Partly cloudy, Partly cloudy  
              2 : Mist + Cloudy, Mist + Broken clouds, Mist+Few clouds, Mist  
              3 : Light Snow, Light RAin + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds  
              4 : Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog  
- temp : temperature in Celsius  
- atemp : "feels like" temperature in Celsius  
- humidity : relative humidity  
- windspeed : wind speed  
- casual : number of non-registered user rentals initiated  
- registered : number of registered user rentals initiated  
- count : number of total rentals  

# 04. EDA

## 01 : 날짜 형태로 변환.

In [4]:
all_data['datetime'] = all_data['datetime'].astype('datetime64')
all_data.head()

## 02. 날짜 데이터를 통해서 년도, 월, 시간 추가.

In [5]:
# 시간, 년 추가.
all_data['year'] = all_data['datetime'].dt.year
all_data['hour'] = all_data['datetime'].dt.hour
all_data['month'] = all_data['datetime'].dt.month

In [6]:
all_data.head()

In [7]:
all_data['holiday'].value_counts().reset_index()
# 대부분 휴일이 아니고, 휴일은 전체 500개의 행 정도 된다.

In [8]:
all_data['workingday'].value_counts().reset_index()
# 주말, 휴일 제외 5514개의 행 정도 된다.

## 03. 그래프 그리기

### 03-1 year별 bke rental 수

In [40]:
all_data['year'].unique()

In [41]:
# 년도별 bike rental 수
plt.figure(figsize=(20,12))
sns.boxplot(all_data['year'],all_data['count'])

In [10]:
# 2012년에 2011년보다 자전거를 많이 대여한 것을 알 수 있다.

### 03-2 시간별 bike rental 수

In [45]:
all_data['hour'].value_counts()

In [47]:
plt.figure(figsize=(20,12))
sns.boxplot(all_data['hour'],all_data['count'])

In [13]:
# 시간대 별도 자전거 대여한 수 차이가 많이나는 것을 확인할 수 있다.
# 새벽에는 많이 타지 않고, 출근길 8, 퇴근길 17,18에 많이 타는 것을 알 수 있다.

### 03-3 windspeed 분포 파악

In [50]:
all_data['windspeed'].describe()

In [48]:
plt.hist(all_data['windspeed'])

In [51]:
# 대부분의 값이 20 이내의 값들로 분포합니다.

### 03-04 계절별 bike rental 수

In [57]:
# 계절은 어떨까?
plt.figure(figsize=(20,15))
sns.boxplot(all_data['season'],all_data['count'])

In [58]:
# 계절별로 count 수 차이가 조금씩 있다고 볼 수 있습니다.

### 03-5 day별로 bike rental 수

In [59]:
# day는 괜찮을까?
all_data['day'] = all_data['datetime'].dt.day

In [60]:
plt.figure(figsize=(20,15))
sns.boxplot(all_data['day'],all_data['count'])
# 큰 차이는 없어 좋은 feature는 아닌 것 같다.
# 1:19만 추출이 가능한가?

In [61]:
# 일별로 봤을 때는 큰 차이는 없고, 20~31일 같은 경우 데이터가 없기 때문에 그래프에 표시되지 않습니다.

### 03-6 holiday별 bike rental 수

In [64]:
plt.figure(figsize=(20,15))
sns.boxplot(all_data['holiday'],all_data['count'])

In [65]:
# 전체적인 분포는 차이가 없어 보이지만, holiday가 0일 때 outlier가 굉장히 많다.

### 03-7 workingday별 bike rental 수

In [66]:
plt.figure(figsize=(10,10))
sns.boxplot(all_data['workingday'],all_data['count'])

In [67]:
# workingday 별도 큰 차이가 없는 것을 확인할 수 있습니다.

### 03-8 weekday 파생변수 만들기

In [68]:
all_data['weekday'] = all_data['datetime'].dt.weekday
all_data

In [69]:
# 요일은 상관 있을까?
plt.figure(figsize=(20,15))
sns.boxplot(all_data['weekday'],all_data['count'])

# 큰 차이는 없어보인다.

In [70]:
# 차이가 조금 있는 것을 확인할 수 있고, 6(일요일) 같은 경우 상대적으로 다른 요일보다 낮은 것을 확인할 수 있습니다.

### 03-9 월별 bike rental 수

In [71]:
plt.figure(figsize=(20,15))
sns.boxplot(all_data['month'],all_data['count'])

In [72]:
#  평균적으로 5~10 큰 차이를 보여주지 않는다.

### 03-10 weather별 bike rental 수

In [74]:
sns.boxplot(all_data['weather'],all_data['count'])

In [76]:
# all_data[all_data['weather']==4] = all_data[all_data['weather']==4].replace({'weather':4},{'weather':3})
# 데이터 분포가 많이 다른 것을 확인할 수 있으며, weather이 4인 경우에는 데이터가 매우 적은 것을 알 수 있습니다.

In [77]:
all_data['weather'].unique()

In [78]:
all_data[all_data['weather']==4]

### 03-11 분석에 필요한 데이터만 추출

In [79]:
all_data2=all_data.drop(['month','datetime','casual','registered','count','day'],axis=1)
all_data2

### 03-12 target 변수의 분포 파악

In [80]:
sns.distplot(train['count'])

- 값이 매우 0 쪽으로 치우쳐져 있는 것을 확인할 수 있다.
- 이런 경우 sqrt, log 등으로 변환 후 분석을 진행하는 것이 효율적이다.
- log를 취하면 좋은점
  - 분포 자체가 넓으면 튀는 값이 많음. -> y의 범위가 줄어들어서 튀는 값을 제어해준다.
  - 분포가 정규분포의 형태로 조금 더 변형되고, 굉장히 튀는 구간의 데이터가 많이 분포 안으로 속하게 된다.

In [83]:
'''
y칼럼에만 log를 취하는 것.(tree 계열 모델)
outlier들을 모델이 파악을 합니다. -> test에서도 outlier가 있을 확률이 높습니다. -> 예측을 거의 하지 못합니다.
문제 : ex) test -> 100인데, train에 800~900이랑 패턴이 비슷하면 800~900으로 예측해버림.
'''
# log 취해주기.
sns.distplot(np.log(train['count']))

In [84]:
# 조금 더 분포가 정규분포의 형태로 띄게 되며, 튀는 값이 적어진 것을 확인할 수 있습니다.

## 04. Modeling

### 04-1 train, test 분리

In [85]:
train2=all_data2[:len(train)]
test2 = all_data2[len(train):]

In [89]:
# catboost : 범주형 데이터가 많을 경우 효율적인 boosting 계열의 모델.
# 처음으로 iterations(반복수) : 3000으로 설정,
# learing_rate(학습률) : 0.01로 설정.
# verbose: 500, 500번 배수로 분석 결과를 도출.

# 라이브러리
from catboost import CatBoostRegressor

# 모델 설정.
cat = CatBoostRegressor(learning_rate=0.01,random_state=42,iterations=3000,verbose=500)

# 모델 학습.
cat.fit(train2,np.log(train['count']))

# 모델 예측.
result = cat.predict(test2)

In [90]:
# all_data로 묶는 다면

In [91]:
# 이 대회에서는 데이터의 특징 때문에 train 월 초 중만 존재 -> test는 월말만 존재. 그래서 모델이 잘 적합하지 못한다.
# holiday. 월초, 월중에 들어간 공휴일과 월말의 공휴일은 다르다.(미국)

In [92]:
sub['count'] = np.exp(result)
sub

In [101]:
# post processing.

#미국 현충일
sub.iloc[1258:1269, 1]= sub.iloc[1258:1269, 1]*0.5
sub.iloc[4492:4515, 1]= sub.iloc[4492:4515, 1]*0.5

#크리스마스 이브
sub.iloc[6308:6330, 1]= sub.iloc[6308:6330, 1]*0.5
sub.iloc[3041:3063, 1]= sub.iloc[3041:3063, 1]*0.5

#크리스마스
sub.iloc[6332:6354, 1]= sub.iloc[6332:6354, 1]*0.5
sub.iloc[3065:3087, 1]= sub.iloc[3065:3087, 1]*0.5

#추수감사절
sub.iloc[5992:6015, 1]= sub.iloc[5992:6015, 1]*0.5
sub.iloc[2771:2794, 1]= sub.iloc[2771:2794, 1]*0.5

In [102]:
sub.to_csv('sub.csv',index=False)

In [103]:
# private leaderboard 4등 위치